In [3]:
import re
import pandas as pd
np = pd.np
np.norm = np.linalg.norm
from datetime import datetime, date
import json
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

from sklearn.feature_extraction.text import TfidfVectorizer  # equivalent to TFIDFTransformer(CountVectorizer())
from django.db.models import Sum

# from pacs.models import CampaignDetail


ValueError: Attempted relative import in non-package

# Django
This is how you join CampaignDetail & WorkingTransactions  
and aggregate the WorkingTransactions.amount

In [4]:
qs = CampaignDetail.objects.annotate(net_amount=Sum('workingtransactions__amount')).values().all()



NameError: name 'CampaignDetail' is not defined

Convert a Django Queryset into a Pandas DataFrame

In [ ]:
df = pd.DataFrame.from_records(qs)

# What if you only want the positive transactions summed
qs_pos = CampaignDetail.objects.filter(workingtransactions__amount__gt=0)
qs_pos = qs_pos.annotate(pos_amount=Sum('workingtransactions__amount'))

# this is how you do a join on a pandas dataframe
df_pos = df.join(pd.DataFrame.from_records(qs_pos.values('pos_amount').all())['pos_amount'])

# but what if I just inserted a new column into the dataframe with the values I want
df['pos_amount'] = pd.DataFrame.from_records(qs_pos.values('pos_amount').all())['pos_amount']

# Did all the rows get inserted in the right place (are the indices still alligned)
print((df == df2))
print((df == df2.mean())
# it turns out that a NaN is not equal to a NaN
# any operation involving a NaN returns a NaN
# and NaN (like None) always evaluates to False
print((df == df2.mean()) + df.isnull().mean())

# get rid of rows without a committee name (from transactions with filer_id not in CampaignDetail)
df = df[df.committee_name.astype(bool)].copy()
df = df.set_index(df.committee_name)

# lets do it all again for negative transaction amounts
qs_neg = CampaignDetail.objects.filter(workingtransactions__amount__lt=0)
qs_neg = qs_neg.annotate(neg_amount=Sum('workingtransactions__amount'))
df = df.join(pd.DataFrame.from_records(qs_pos.values('neg_amount').all())['neg_amount'])


# Can we create a directed graph of financial transactions between committees?
# are the payee_committee_ids the same as "filer_id"?
filer_id = set(pd.DataFrame.from_records(WorkingTransactions.objects.values(
               'filer_id').all()).dropna().values.T[0])
payee_id = set(pd.DataFrame.from_records(WorkingTransactions.objects.values(
               'contributor_payee_committee_id').all()).dropna().values.T[0])
print(len(payee_id.intersection(filer_id)) * 1. / len(filer_id))

# That's good enough for government work

